In [1]:
import mido
import os
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.utils import np_utils


Using TensorFlow backend.
/Users/StephanieBrito/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 8

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)
    
    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    print(network_input)
    
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    #model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    #model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    #model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

In [3]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=10, batch_size=64, callbacks=callbacks_list)
    return model

In [4]:
def get_notes():
    mid = MidiFile('Data/lloraras.mid')  
    notes = []
    for msg in mid:
#         if msg.type == 'program_change':
#             print(msg)
        if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
            data = msg.bytes()
            notes.append(data[1])
    return notes

In [5]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()
    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    return train(model, network_input, network_output)


In [6]:
model = train_network()

[[[0.47826087]
  [0.2173913 ]
  [0.52173913]
  ...
  [0.2826087 ]
  [0.47826087]
  [0.2826087 ]]

 [[0.2173913 ]
  [0.52173913]
  [0.52173913]
  ...
  [0.47826087]
  [0.2826087 ]
  [0.36956522]]

 [[0.52173913]
  [0.52173913]
  [0.2173913 ]
  ...
  [0.2826087 ]
  [0.36956522]
  [0.36956522]]

 ...

 [[0.13043478]
  [0.06521739]
  [0.04347826]
  ...
  [0.02173913]
  [0.06521739]
  [0.04347826]]

 [[0.06521739]
  [0.04347826]
  [0.10869565]
  ...
  [0.06521739]
  [0.04347826]
  [0.        ]]

 [[0.04347826]
  [0.10869565]
  [0.08695652]
  ...
  [0.04347826]
  [0.        ]
  [0.02173913]]]
Epoch 1/10
4868/4868 [==============================] - 42s 9ms/step - loss: 3.3972
Epoch 2/10
4868/4868 [==============================] - 49s 10ms/step - loss: 3.2256
Epoch 3/10
4868/4868 [==============================] - 62s 13ms/step - loss: 3.1442
Epoch 4/10
4868/4868 [==============================] - 57s 12ms/step - loss: 3.1152
Epoch 5/10
4868/4868 [==============================] - 55s 11ms/st

In [10]:
notes = get_notes()
    # get amount of pitch names
n_vocab = len(set(notes))

network_input, network_output = prepare_sequences(notes, n_vocab)

prediction = model.predict(np.array(network_input))
prediction = np.squeeze(prediction)
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]


[[[0.47826087]
  [0.2173913 ]
  [0.52173913]
  ...
  [0.2826087 ]
  [0.47826087]
  [0.2826087 ]]

 [[0.2173913 ]
  [0.52173913]
  [0.52173913]
  ...
  [0.47826087]
  [0.2826087 ]
  [0.36956522]]

 [[0.52173913]
  [0.52173913]
  [0.2173913 ]
  ...
  [0.2826087 ]
  [0.36956522]
  [0.36956522]]

 ...

 [[0.13043478]
  [0.06521739]
  [0.04347826]
  ...
  [0.02173913]
  [0.06521739]
  [0.04347826]]

 [[0.06521739]
  [0.04347826]
  [0.10869565]
  ...
  [0.06521739]
  [0.04347826]
  [0.        ]]

 [[0.04347826]
  [0.10869565]
  [0.08695652]
  ...
  [0.04347826]
  [0.        ]
  [0.02173913]]]


/Users/StephanieBrito/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [11]:
mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('10_epoch_8_note_seq_CE_loss.mid')

In [ ]:
'''
Notes: there's some weird shape errors here... will abandon this for now...
'''